In [1]:
pip install torch torchvision transformers tabulate streamlit


                                              0.0/8.1 MB ? eta -:--:--
     ------------------                       3.7/8.1 MB 79.1 MB/s eta 0:00:01
     ---------------------------------------  8.1/8.1 MB 103.5 MB/s eta 0:00:01
     ---------------------------------------- 8.1/8.1 MB 74.1 MB/s eta 0:00:00
                                              0.0/471.5 kB ? eta -:--:--
     ------------------------------------- 471.5/471.5 kB 28.8 MB/s eta 0:00:00
                                              0.0/97.9 kB ? eta -:--:--
     ---------------------------------------- 97.9/97.9 kB ? eta 0:00:00
                                              0.0/422.5 kB ? eta -:--:--
     ---------------------------------------- 422.5/422.5 kB ? eta 0:00:00
                                              0.0/21.5 MB ? eta -:--:--
     -------                                 4.3/21.5 MB 137.7 MB/s eta 0:00:01
     -----------------                       9.7/21.5 MB 124.1 MB/s eta 0:00:01
     -------


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\Bryon\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from transformers import pipeline

# Load the pipeline with the fine-tuned model for image classification
pipe = pipeline("image-classification", model="rafalosa/diabetic-retinopathy-224-procnorm-vit")


C:\Users\Bryon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png;*.jpeg")])
    if file_path:
        display_image(file_path)
        evaluate_model(file_path)

def resize_with_aspect_ratio(image, max_size):
    width, height = image.size
    if width > height:
        new_width = max_size
        new_height = int(height * max_size / width)
    else:
        new_height = max_size
        new_width = int(width * max_size / height)
    return image.resize((new_width, new_height), Image.ANTIALIAS)

def display_image(image_path):
    image = Image.open(image_path)
    # Resize the image for display, preserving aspect ratio
    image = resize_with_aspect_ratio(image, 300)
    photo = ImageTk.PhotoImage(image)
    image_label.config(image=photo)
    image_label.image = photo

def evaluate_model(image_path):
    results = pipe(image_path)

    # Update class labels to full names and capitalize
    for result in results:
        result['label'] = result['label'].capitalize().replace("dr", "Diabetic Retinopathy")

    # Create table data with headers and rows
    table_data = [("Class", "Confidence")] + [(result['label'], "{:.2f}".format(result['score'])) for result in results]

    # Remove any existing table entries
    for widget in table_frame.winfo_children():
        widget.destroy()

    # Insert data into the table
    for i, (header, data) in enumerate(table_data):
        bold_font = ('Arial', 12, 'bold') if i == 0 else ('Arial', 12)

        e = tk.Entry(table_frame, width=25, fg='black', font=bold_font, bd=0)  # Set border width (bd) to 0
        e.grid(row=i, column=0, padx=5, pady=5)
        e.insert(tk.END, header)
        e.config(state='readonly')

        e = tk.Entry(table_frame, width=15, fg='black', font=bold_font, bd=0)  # Set border width (bd) to 0
        e.grid(row=i, column=1, padx=5, pady=5)
        e.insert(tk.END, data)
        e.config(state='readonly')


In [32]:

# Create the main application window
root = tk.Tk()
root.title("Diabetic Retinopathy Detection")

# Set the initial window size (width x height)
root.geometry("800x700")
# Set the minimum window size (width x height)
root.minsize(500, 650)

# Create the image label
image_label = tk.Label(root)
image_label.pack(pady=15)

# Create the "Upload Image" button
upload_button = tk.Button(root, text="Upload Image", command=open_image)
upload_button.pack(pady=10)
"""
# Create a label to show the model's predictions
result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, font=("Arial", 12))
result_label.pack()
"""
# Create a frame for the table
table_frame = tk.Frame(root)
table_frame.pack(pady=10)

# Start the main event loop
root.mainloop()

C:\Users\Bryon\AppData\Local\Temp\ipykernel_18616\2511145971.py:15: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  return image.resize((new_width, new_height), Image.ANTIALIAS)
